In [1]:
from sklearn.model_selection import train_test_split
from Source.code import *
from sklearn import preprocessing
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot
%matplotlib inline
from numpy import dtype, sin, sqrt, asarray, mean, std
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
#print(le_arq("./Data/telecom_users.csv"))
df = le_arq("./Data/telecom_users.csv")
le = preprocessing.LabelEncoder()
#print(df.head(10))

In [3]:
df = df.drop('IDCliente', 1)
for i in df:
    if(dtype(df[i]) == 'object'):
        df[i] = le.fit_transform(df[i])
df.head(10)

/tmp/ipykernel_14247/6696580.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('IDCliente', 1)


NameError: name 'dtype' is not defined

In [ ]:
Churn = df[['Churn']]
df = df.drop('Churn', 1)

/tmp/ipykernel_13186/705538376.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('Churn', 1)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df.values, Churn.values, random_state=42, test_size=0.3)

In [ ]:
#training a logistics regression model
logmodel = LogisticRegression()
logmodel.fit(X_train, Y_train)
predictions = logmodel.predict(X_test)
print("Accuracy = " + str(accuracy_score(Y_test, predictions)))

NameError: name 'X_train' is not defined

In [ ]:
#defining various steps required for the genetic algorithm
def initilization_of_population(size, n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat, dtype=np.bool)
        chromosome[:int(0.3*n_feat)] = False
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population


def fitness_score(population):
    scores = []
    for chromosome in population:
        logmodel.fit(X_train.iloc[:, chromosome], Y_train)
        predictions = logmodel.predict(X_test.iloc[:, chromosome])
        scores.append(accuracy_score(Y_test, predictions))
    scores, population = np.array(scores), np.array(population)
    inds = np.argsort(scores)
    return list(scores[inds][::-1]), list(population[inds, :][::-1])


def selection(pop_after_fit, n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen


def crossover(pop_after_sel):
    population_nextgen = pop_after_sel
    for i in range(len(pop_after_sel)):
        child = pop_after_sel[i]
        child[3:7] = pop_after_sel[(i+1) % len(pop_after_sel)][3:7]
        population_nextgen.append(child)
    return population_nextgen


def mutation(pop_after_cross, mutation_rate):
    population_nextgen = []
    for i in range(0, len(pop_after_cross)):
        chromosome = pop_after_cross[i]
        for j in range(len(chromosome)):
            if random.random() < mutation_rate:
                chromosome[j] = not chromosome[j]
        population_nextgen.append(chromosome)
    #print(population_nextgen)
    return population_nextgen


def generations(size, n_feat, n_parents, mutation_rate, n_gen, X_train,
                X_test, y_train, y_test):
    best_chromo = []
    best_score = []
    population_nextgen = initilization_of_population(size, n_feat)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen)
        print(scores[:2])
        pop_after_sel = selection(pop_after_fit, n_parents)
        pop_after_cross = crossover(pop_after_sel)
        population_nextgen = mutation(pop_after_cross, mutation_rate)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo, best_score


In [ ]:
chromo, score = generations(size=200, n_feat=19, n_parents=100, mutation_rate=0.10,
                            n_gen=38, X_train=X_train, X_test=X_test, y_train=Y_train, y_test=Y_test)
logmodel.fit(X_train.iloc[:, chromo[-1]], Y_train)
predictions = logmodel.predict(X_test.iloc[:, chromo[-1]])
print("Accuracy score after genetic algorithm is= " + str(accuracy_score(Y_test, predictions)))


/tmp/ipykernel_13186/1270278624.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  chromosome = np.ones(n_feat, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable